In [29]:
# Import Numpy & PyTorch
import numpy as np
import torch

## Linear Regression Model using PyTorch built-ins

Let's re-implement the same model using some built-in functions and classes from PyTorch.

And now using two different targets: Apples and Oranges

In [30]:
# Imports
import torch.nn as nn

In [31]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119]], dtype='float32')

In [32]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples. We'll also create a DataLoader, to split the data into batches while training. It also provides other utilities like shuffling and sampling.

In [33]:
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader

In [34]:
# Define dataset
#The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). 
#It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.
train_ds = TensorDataset(inputs, targets)
train_ds[0:5]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

In [35]:
# Define data loader
# DataLoader, which can split the data into batches of a predefined size while training. 
# It also provides other utilities like shuffling and random sampling of the data
from torch.utils.data import DataLoader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

### nn.Linear
Instead of initializing the weights & biases manually, we can define the model using `nn.Linear`.

In [36]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.5261, -0.3157, -0.2737],
        [-0.4500,  0.4727,  0.2999]], requires_grad=True)
Parameter containing:
tensor([ 0.3621, -0.1078], requires_grad=True)


### Optimizer
Instead of manually manipulating the weights & biases using gradients, we can use the optimizer `optim.SGD`.

In [37]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

### Loss Function
Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [38]:
# Import nn.functional
import torch.nn.functional as F

In [39]:
# Define loss function
loss_fn = F.mse_loss

In [40]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(6733.8394, grad_fn=<MseLossBackward>)


### Train the model

We are ready to train the model now. We can define a utility function `fit` which trains the model for a given number of epochs.

In [41]:
# Define a utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
  for epoch in range(num_epochs):
    for xb,yb in train_dl:
      # Generate predictions
      pred = model(xb)
      loss = loss_fn(pred, yb)
      # Perform gradient descent
      loss.backward()
      opt.step()
      opt.zero_grad()
    if (epoch+1) % 10 == 0:
      print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [42]:
# Train the model for 100 epochs
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 844.4696
Epoch [20/100], Loss: 916.7046
Epoch [30/100], Loss: 302.1601
Epoch [40/100], Loss: 180.2492
Epoch [50/100], Loss: 136.4704
Epoch [60/100], Loss: 77.7845
Epoch [70/100], Loss: 68.7485
Epoch [80/100], Loss: 75.2116
Epoch [90/100], Loss: 59.7060
Epoch [100/100], Loss: 32.7858


In [43]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 59.2881,  70.5868],
        [ 80.0346,  98.5606],
        [119.6239, 136.5539],
        [ 33.1819,  39.0991],
        [ 91.1941, 114.2861],
        [ 59.2881,  70.5868],
        [ 80.0346,  98.5606],
        [119.6239, 136.5539],
        [ 33.1819,  39.0991],
        [ 91.1941, 114.2861],
        [ 59.2881,  70.5868],
        [ 80.0346,  98.5606],
        [119.6239, 136.5539],
        [ 33.1819,  39.0991],
        [ 91.1941, 114.2861]], grad_fn=<AddmmBackward>)

In [44]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Now we can define the model, optimizer and loss function exactly as before.

In [46]:
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 13.3526
Epoch [20/100], Loss: 19.5804
Epoch [30/100], Loss: 10.3623
Epoch [40/100], Loss: 15.9731
Epoch [50/100], Loss: 5.2628
Epoch [60/100], Loss: 13.5574
Epoch [70/100], Loss: 4.3861
Epoch [80/100], Loss: 1.7320
Epoch [90/100], Loss: 10.8259
Epoch [100/100], Loss: 7.2889
